In [2]:
%%cql select * from music.tracks_by_album limit 5

album_title,album_year,track_number,album_genre,performer,track_title
Duos For Violin and Cello,2000,1,Classical,Nigel Kennedy,Sonata for Violin and Cello - Allegro
Duos For Violin and Cello,2000,2,Classical,Nigel Kennedy,Sonata for Violin and Cello - Tres vif
Duos For Violin and Cello,2000,3,Classical,Nigel Kennedy,Sonata for Violin and Cello - Lent
Duos For Violin and Cello,2000,4,Classical,Nigel Kennedy,"Sonata for Violin and Cello - Vif, avec entrain"
Duos For Violin and Cello,2000,5,Classical,Nigel Kennedy,Passacaglia


### Create a SQL Context
 - The sqlContext already exists

### Create a dataframe on a cassandra table

In [5]:
df = sqlContext.read\
   .format("org.apache.spark.sql.cassandra")\
   .options(keyspace = "music", table = "tracks_by_album")\
   .load()	

### Explain the query plan and view some data

In [6]:
df.printSchema()

root
 |-- album_title: string (nullable = true)
 |-- album_year: integer (nullable = true)
 |-- track_number: integer (nullable = true)
 |-- album_genre: string (nullable = true)
 |-- performer: string (nullable = true)
 |-- track_title: string (nullable = true)



In [7]:
df.explain()

PhysicalRDD [album_title#0,album_year#1,track_number#2,album_genre#3,performer#4,track_title#5], MapPartitionsRDD[2] at executedPlan at NativeMethodAccessorImpl.java:-2



In [8]:
df.show()

+--------------------+----------+------------+-----------+-------------+--------------------+
|         album_title|album_year|track_number|album_genre|    performer|         track_title|
+--------------------+----------+------------+-----------+-------------+--------------------+
|Duos For Violin a...|      2000|           1|  Classical|Nigel Kennedy|Sonata for Violin...|
|Duos For Violin a...|      2000|           2|  Classical|Nigel Kennedy|Sonata for Violin...|
|Duos For Violin a...|      2000|           3|  Classical|Nigel Kennedy|Sonata for Violin...|
|Duos For Violin a...|      2000|           4|  Classical|Nigel Kennedy|Sonata for Violin...|
|Duos For Violin a...|      2000|           5|  Classical|Nigel Kennedy|         Passacaglia|
|Duos For Violin a...|      2000|           6|  Classical|Nigel Kennedy|Duo for Violin an...|
|Duos For Violin a...|      2000|           7|  Classical|Nigel Kennedy|Duo for Violin an...|
|Duos For Violin a...|      2000|           8|  Classical|Ni

In [10]:
df.select("album_year").distinct().show()

+----------+
|album_year|
+----------+
|      1952|
|      1956|
|      1957|
|      1958|
|      1959|
|      1960|
|      1961|
|      1962|
|      1963|
|      1964|
|      1965|
|      1966|
|      1967|
|      1968|
|      1969|
|      1970|
|      1971|
|      1972|
|      1973|
|      1974|
+----------+



In [11]:
df.groupBy("album_year").count().show()

+----------+-----+
|album_year|count|
+----------+-----+
|      1952|   10|
|      1956|   53|
|      1957|   21|
|      1958|   31|
|      1959|   28|
|      1960|   42|
|      1961|   86|
|      1962|  129|
|      1963|  101|
|      1964|  147|
|      1965|  203|
|      1966|   95|
|      1967|  244|
|      1968|  243|
|      1969|  326|
|      1970|  379|
|      1971|  420|
|      1972|  262|
|      1973|  443|
|      1974|  489|
+----------+-----+



### Group By Decade
You can use various spark sql functions.  Let's use *floor*.

In [15]:
from pyspark.sql.functions import *

In [17]:
df.groupBy(floor(col("album_year") / 10) * 10).count().show()

+-------------------------------+-----+
|(FLOOR((album_year / 10)) * 10)|count|
+-------------------------------+-----+
|                         2000.0| 9497|
|                         1950.0|  143|
|                         1960.0| 1616|
|                         1970.0| 4346|
|                         1980.0| 6390|
|                         1990.0|14759|
+-------------------------------+-----+



### Clean it up

In [21]:
tmp = df.groupBy((floor(col("album_year") / 10) * 10).cast("int").alias("decade")).count()
tmp.show()

+------+-----+
|decade|count|
+------+-----+
|  1950|  143|
|  1960| 1616|
|  1970| 4346|
|  1980| 6390|
|  1990|14759|
|  2000| 9497|
+------+-----+



In [25]:
count_by_decade = tmp.select(col("decade"), col("count").alias("album_count"))
count_by_decade.show()

+------+-----------+
|decade|album_count|
+------+-----------+
|  1950|        143|
|  1960|       1616|
|  1970|       4346|
|  1980|       6390|
|  1990|      14759|
|  2000|       9497|
+------+-----------+



### Save to a new table

In [30]:
count_by_decade.write.format("org.apache.spark.sql.cassandra")\
.options(table = "albums_by_decade", keyspace = "steve")\
.mode('overwrite')\
.save()